# Programming Agent Memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a MemGPT client 

In [2]:
from letta import create_client 

client = create_client() 

/Users/azinasgarian/miniconda3/envs/mem/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Letta.letta.server.server - INFO - Creating sqlite engine sqlite:////Users/azinasgarian/.letta/sqlite.db


In [3]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig


# Set the default llm config
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

# Set the default embedding config
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002"))

## Section 1: Memory Blocks 

### Understanding ChatMemory

In [4]:
from letta.schemas.memory import ChatMemory  

In [5]:
chat_memory = ChatMemory(
    human="Name: Bob", 
    persona="You are a helpful assistant"
)

In [6]:
chat_memory.list_block_labels()

['persona', 'human']

In [7]:
chat_memory.get_block("human")

Block(value='Name: Bob', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-38d6ed0c-da95-43eb-9854-1b06cfc28d87', organization_id=None, created_by_id=None, last_updated_by_id=None)

In [8]:
import inspect

In [9]:
print(inspect.getsource(chat_memory.core_memory_append))

    def core_memory_append(agent_state: "AgentState", label: str, content: str) -> Optional[str]:  # type: ignore
        """
        Append to the contents of core memory.

        Args:
            label (str): Section of the memory to be edited (persona or human).
            content (str): Content to write to the memory. All unicode (including emojis) are supported.

        Returns:
            Optional[str]: None is always returned as this function does not produce a response.
        """
        current_value = str(agent_state.memory.get_block(label).value)
        new_value = current_value + "\n" + str(content)
        agent_state.memory.update_block_value(label=label, value=new_value)
        return None



#### Context compilation 

In [10]:
chat_memory.get_prompt_template()

'{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}'

In [11]:
chat_memory.compile()

'<persona characters="27/5000">\nYou are a helpful assistant\n</persona>\n<human characters="9/5000">\nName: Bob\n</human>'

## Section 2: Defining a custom memory module

In this section, we're going to define a custom memory module called `TaskMemory`. This module will be used to store and manage tasks for a user in the core memory along with the human and persona. We'll also implement 2 functions to manage the task queue called `task_queue_push` and `task_queue_pop`.

### Defining a memory module


In [12]:
from letta.schemas.memory import ChatMemory
from letta.schemas.block import Block
from typing import Optional, List
import json

In [13]:
class TaskMemory(ChatMemory): 

    def __init__(self, human: str, persona: str, tasks: str): 
        super().__init__(human=human, persona=persona, limit=2000) 
        self.set_block(Block(limit=2000, value=json.dumps(tasks), label="tasks"))

    def task_queue_push(self: "Agent", task_description: str):
        """
        Push to a task queue stored in core memory. 

        Args:
            task_description (str): A description of the next task you must accomplish. 
            
        Returns:
            Optional[str]: None is always returned as this function 
            does not produce a response.
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        tasks.append(task_description)
        self.memory.update_block_value("tasks", json.dumps(tasks))
        return None

    def task_queue_pop(self: "Agent"):
        """
        Get the next task from the task queue 
 
        Returns:
            Optional[str]: The description of the task popped from the 
            queue, if there are still tasks in queue. Otherwise, returns
            None (the task queue is empty)
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        if len(tasks) == 0: 
            return None
        task = tasks[0]
        print("CURRENT TASKS: ", tasks)
        self.memory.update_block_value("tasks", json.dumps(tasks[1:]))
        return task

### Creating an agent with custom `TaskMemory`

In [14]:
task_agent_name = "task_agent"

task_agent_state = client.create_agent(
    name=task_agent_name, 
    system = open("task_queue_system_prompt.txt", "r").read(),
    memory=TaskMemory(
        human="My name is Azin", 
        persona="You are an agent that must clear its tasks.", 
        tasks=[]
    )
)

In [15]:
message = "Task 1: Explain my name, Task 2: Tell me a story about my name in less than 100 words"

In [16]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message=message
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 22, 15, 11, 412864, tzinfo=datetime.timezone.utc) updated_at=None id='message-9557aa75-117b-430a-ab4b-4092f5d378c2' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="Adding user's name to core memory for personalization.")] organization_id=None agent_id='agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_jfl7Ez0jYeRPhyVs8yugPEo3', function=Function(arguments='{\n  "label": "human",\n  "content": "Azin",\n  "request_heartbeat": true\n}', name='core_memory_append'), type='function')] tool_call_id=None step_id=None
Letta.agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb - INFO - Request to call function core_

LettaResponse(messages=[ReasoningMessage(id='message-9557aa75-117b-430a-ab4b-4092f5d378c2', date=datetime.datetime(2025, 2, 10, 22, 15, 11, 412864, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="Adding user's name to core memory for personalization."), ToolCallMessage(id='message-9557aa75-117b-430a-ab4b-4092f5d378c2', date=datetime.datetime(2025, 2, 10, 22, 15, 11, 412864, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_append', arguments='{\n  "label": "human",\n  "content": "Azin",\n  "request_heartbeat": true\n}', tool_call_id='call_jfl7Ez0jYeRPhyVs8yugPEo3')), ToolReturnMessage(id='message-3a3eb35b-bedd-4833-b3d0-bd9891762821', date=datetime.datetime(2025, 2, 10, 22, 15, 11, 506441, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 05:15:11 PM EST-0500"\n}', status='success', tool_call_id='call_jfl7Ez0jYeRPhyVs8yugPEo3', stdout=None, stderr=None), ReasoningMessage(id='message-b97c7b67-1893-43fd-a5ca-9d1c72dbdd28', date=datetime.datetime(2025, 2, 10, 22, 15, 13, 804367, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Tasks received from the user. Preparing to fulfill them.'), ToolCallMessage(id='message-b97c7b67-1893-43fd-a5ca-9d1c72dbdd28', date=datetime.datetime(2025, 2, 10, 22, 15, 13, 804367, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Your name, Azin, has Persian roots, meaning \\"intelligent\\" or \\"wise.\\" It represents a person with great insight and understanding. Now, let me tell you a short story: Once, there was a wise sage named Azin who traveled the world, sharing knowledge and solving problems. People sought him out, knowing his intelligence could guide them through life\'s challenges."\n}', tool_call_id='call_JAn85xA0WqxUmqjkZhT3Mtva')), ToolReturnMessage(id='message-032f4b7d-81d9-4c2f-b5a4-dbd9a35547a3', date=datetime.datetime(2025, 2, 10, 22, 15, 13, 808743, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 05:15:13 PM EST-0500"\n}', status='success', tool_call_id='call_JAn85xA0WqxUmqjkZhT3Mtva', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=146, prompt_tokens=4642, total_tokens=4788, step_count=2))

In [17]:
task_agent_state.memory.get_blocks()

[Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-13a176c4-04a9-4ee7-8eb0-4c498f86d625', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Block(value='[]', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-267f452b-e779-43e1-b760-561577b57066', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-3ba4155f-b488-4175-ac6e-5e460a27f30e', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

In [18]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 22, 15, 15, 771812, tzinfo=datetime.timezone.utc) updated_at=None id='message-d9a69155-386f-42c6-942d-db549f0d80ad' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User wants me to confirm task completion. I should clarify that tasks are already done.')] organization_id=None agent_id='agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_nXVo2R0wWldyh9xkBj9Iviw2', function=Function(arguments='{\n  "message": "I\'ve completed the tasks you set! If you have more questions or need assistance, feel free to let me know."\n}', name='send_message'), type='function')] tool_call_id=None step_id=None
Letta.age

LettaResponse(messages=[ReasoningMessage(id='message-d9a69155-386f-42c6-942d-db549f0d80ad', date=datetime.datetime(2025, 2, 10, 22, 15, 15, 771812, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User wants me to confirm task completion. I should clarify that tasks are already done.'), ToolCallMessage(id='message-d9a69155-386f-42c6-942d-db549f0d80ad', date=datetime.datetime(2025, 2, 10, 22, 15, 15, 771812, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "I\'ve completed the tasks you set! If you have more questions or need assistance, feel free to let me know."\n}', tool_call_id='call_nXVo2R0wWldyh9xkBj9Iviw2')), ToolReturnMessage(id='message-619a2465-0364-40ca-88fd-47db9bff9534', date=datetime.datetime(2025, 2, 10, 22, 15, 15, 776753, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 05:15:15 PM EST-0500"\n}', status='success', tool_call_id='call_nXVo2R0wWldyh9xkBj9Iviw2', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=59, prompt_tokens=2595, total_tokens=2654, step_count=1))

In [19]:
client.get_core_memory(task_agent_state.id).get_blocks()

[Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-13a176c4-04a9-4ee7-8eb0-4c498f86d625', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Block(value='[]', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-267f452b-e779-43e1-b760-561577b57066', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin\nAzin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-3ba4155f-b488-4175-ac6e-5e460a27f30e', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

> copy the id='block-...' string, from the code cell above "client.get_core_memory...", and then paste into the code cell client.get_block('block-...')

In [20]:
print(client.get_block('cut_and_paste_id_from_above'))

None


In [21]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 22, 15, 17, 838406, tzinfo=datetime.timezone.utc) updated_at=None id='message-c41b8943-93a7-4bd2-9d91-ebaefac5d756' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="User seems to be repeating the request. I'll reiterate that there are no outstanding tasks.")] organization_id=None agent_id='agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_sGSvLDUGETb1mQeDaW2MkDL3', function=Function(arguments='{\n  "message": "All tasks are complete, Azin! If you have anything else in mind or new tasks for me, just let me know!"\n}', name='send_message'), type='function')] tool_call_id=None step_id=None
Letta.agen

LettaResponse(messages=[ReasoningMessage(id='message-c41b8943-93a7-4bd2-9d91-ebaefac5d756', date=datetime.datetime(2025, 2, 10, 22, 15, 17, 838406, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="User seems to be repeating the request. I'll reiterate that there are no outstanding tasks."), ToolCallMessage(id='message-c41b8943-93a7-4bd2-9d91-ebaefac5d756', date=datetime.datetime(2025, 2, 10, 22, 15, 17, 838406, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "All tasks are complete, Azin! If you have anything else in mind or new tasks for me, just let me know!"\n}', tool_call_id='call_sGSvLDUGETb1mQeDaW2MkDL3')), ToolReturnMessage(id='message-b2319799-3001-449d-9ed0-df09e6111ae1', date=datetime.datetime(2025, 2, 10, 22, 15, 17, 844816, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 05:15:17 PM EST-0500"\n}', status='success', tool_call_id='call_sGSvLDUGETb1mQeDaW2MkDL3', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=63, prompt_tokens=2746, total_tokens=2809, step_count=1))

In [22]:
client.get_core_memory(task_agent_state.id).get_blocks()

[Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-13a176c4-04a9-4ee7-8eb0-4c498f86d625', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Block(value='[]', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-267f452b-e779-43e1-b760-561577b57066', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin\nAzin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-3ba4155f-b488-4175-ac6e-5e460a27f30e', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

In [23]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="Where is that memory?"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 22, 15, 19, 725239, tzinfo=datetime.timezone.utc) updated_at=None id='message-c35d55db-205d-4643-a580-5640c35d646c' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="User is curious about memory. I'll explain without revealing too much technical detail.")] organization_id=None agent_id='agent-fcbce9ce-ff80-4b09-a1c0-9fcbc0ee3afb' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_2LnLbbHqQDBgH66KNFklHtGV', function=Function(arguments='{\n  "message": "The memory I have about you is stored internally, allowing me to remember your name, Azin, and provide a more personalized experience. If there\'s something specific you\'d like me to recall or 

LettaResponse(messages=[ReasoningMessage(id='message-c35d55db-205d-4643-a580-5640c35d646c', date=datetime.datetime(2025, 2, 10, 22, 15, 19, 725239, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="User is curious about memory. I'll explain without revealing too much technical detail."), ToolCallMessage(id='message-c35d55db-205d-4643-a580-5640c35d646c', date=datetime.datetime(2025, 2, 10, 22, 15, 19, 725239, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "The memory I have about you is stored internally, allowing me to remember your name, Azin, and provide a more personalized experience. If there\'s something specific you\'d like me to recall or discuss, just let me know!"\n}', tool_call_id='call_2LnLbbHqQDBgH66KNFklHtGV')), ToolReturnMessage(id='message-7f998528-d5a4-464b-abc6-f5f7d62436f3', date=datetime.datetime(2025, 2, 10, 22, 15, 19, 731370, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 05:15:19 PM EST-0500"\n}', status='success', tool_call_id='call_2LnLbbHqQDBgH66KNFklHtGV', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=78, prompt_tokens=2902, total_tokens=2980, step_count=1))

In [24]:
task_agent_state.memory.get_block("persona")

Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-13a176c4-04a9-4ee7-8eb0-4c498f86d625', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)